In [194]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

import json
from dotenv import load_dotenv
import os
import requests
import aiohttp
import asyncio

In [178]:


# Define the scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Add your service account credentials here
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope,)

# Authorize the client
client = gspread.authorize(credentials)

# Open the Google Sheet by its title
sheet = client.open('Essentials').worksheet(title='Expenses')

# Get the data as a list of lists
data = sheet.get_all_values()

# Convert it into a pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

In [181]:
filtered_needed_df = df.iloc[7:]
filtered_needed_df.columns = df.iloc[6].values # changing the column name in the freezed rows
filtered_needed_df = filtered_needed_df[df.iloc[6].values[:7]] # getting only needed columns
filtered_needed_df.columns = ['transaction_date', 'description', 'dollars','salary', 'credits', 'in_hand','debits'] # renaming cols as needed
filtered_needed_df['transaction_date'] = pd.to_datetime(filtered_needed_df['transaction_date'])
filtered_needed_df[['dollars','salary', 'credits', 'in_hand','debits']] = filtered_needed_df[['dollars','salary', 'credits', 'in_hand','debits']].replace('','0.0')

for col in ['dollars','salary', 'credits', 'in_hand','debits']:
    filtered_needed_df[col] = filtered_needed_df[col].str.strip().str.replace(",",'')
    filtered_needed_df[col].loc[filtered_needed_df[col]==''] =  filtered_needed_df[col].loc[filtered_needed_df[col]==''].replace('','0.0')
    filtered_needed_df[col] =filtered_needed_df[col].astype('float')

C:\Users\sanja\AppData\Local\Temp\ipykernel_13620\322000533.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  filtered_needed_df[col].loc[filtered_needed_df[col]==''] =  filtered_needed_df[col].loc[filtered_needed_df[col]==''].replace('',

In [244]:
mod_data = filtered_needed_df.copy()
mod_data['year'] = mod_data.transaction_date.dt.year
mod_data['month'] = mod_data.transaction_date.dt.month
mod_data['st_of_month'] = pd.to_datetime(mod_data.transaction_date.dt.year.astype('str')+'-'+mod_data.transaction_date.dt.month.astype('str')+'-01')

def get_spent_percentage(group, gby_cols, dollar_rate = 300, calc_cols = ['dollars','credits','debits', 'in_hand'] ):

    total_bank_credits = group[group.dollars==0].credits.sum()
    total_bank_debits = group.debits.sum()

    total_inhand_credits = group[(group.in_hand>0)&(group.debits==0)].in_hand.sum()
    total_inhand_debits = group[(group.in_hand<0)&(group.credits==0)].in_hand.sum()

    total_dollar_credits = group[group.dollars>0].dollars.sum()*dollar_rate
    total_dollar_debits = group[group.dollars<0].dollars.sum()*dollar_rate

    total_credits = (total_bank_credits+total_inhand_credits+total_dollar_credits)
    total_debits = (total_inhand_debits+total_bank_debits+total_dollar_debits)

    spent_percentage = total_debits/total_credits

    out_group = group[gby_cols].drop_duplicates().copy()
    out_group['spent_percent'] = spent_percentage
    out_group['total_debits'] = total_debits
    out_group['total_credits'] = total_credits

    return out_group

TODAY = datetime.datetime.today()
START_OF_CURRENT_MONTH = pd.to_datetime(f'{TODAY.year}-{TODAY.month}-01')

mod_data = mod_data[mod_data.transaction_date<START_OF_CURRENT_MONTH].groupby(['st_of_month']).apply(lambda group: get_spent_percentage(group=group, gby_cols=['st_of_month'])).reset_index(drop=True)

C:\Users\sanja\AppData\Local\Temp\ipykernel_13620\249524142.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mod_data = mod_data[mod_data.transaction_date<START_OF_CURRENT_MONTH].groupby(['st_of_month']).apply(lambda group: get_spent_percentage(group=group, gby_cols=['st_of_month'])).reset_index(drop=True)


In [232]:
mod_data 

,st_of_month,spent_percent,total_debits,total_credits
0,2021-09-01,0.039600,198.0,5000.0
1,2021-10-01,1.117326,41788.0,37400.0
2,2021-11-01,0.569786,13333.0,23400.0
3,2021-12-01,1.007632,42119.0,41800.0
4,2022-01-01,1.207215,34466.0,28550.0
5,2022-02-01,0.295722,26198.0,88590.0
6,2022-03-01,1.141048,363515.0,318580.0
7,2022-04-01,1.082982,193177.0,178375.0
8,2022-05-01,0.980203,122395.0,124867.0
9,2022-06-01,0.139766,21800.0,155975.0


In [248]:
mod_data.sort_values('st_of_month').tail(3)

,st_of_month,spent_percent,total_debits,total_credits
33,2024-06-01,-0.009609,-3200.0,333010.0
34,2024-07-01,0.210208,78292.0,372450.0
35,2024-08-01,0.079006,23588.0,298560.0


In [250]:
if START_OF_CURRENT_MONTH.month==1:
    YEAR = START_OF_CURRENT_MONTH.year-1
    MONTH = 12
else:
    YEAR = START_OF_CURRENT_MONTH.year
    MONTH  = START_OF_CURRENT_MONTH.month

PAST_2_MONTHS_ST = pd.to_datetime('2024-08-01')-pd.to_timedelta(9, unit='W')

LAST_MONTH = mod_data.sort_values('st_of_month').tail(1)

LAST_3_MONTHS = mod_data.sort_values('st_of_month').tail(3)

In [269]:
WA_analytics = f"Last month=> Spent %: {np.round(LAST_MONTH['spent_percent'].values[0],4)}, Total Credits: LKR {LAST_MONTH['total_credits'].values[0]}, Total Debits: LKR {LAST_MONTH['total_debits'].values[0]}"

In [270]:

# --------------------------------------------------------------
# Load environment variables
# --------------------------------------------------------------

load_dotenv()
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
RECIPIENT_WAID = os.getenv("RECIPIENT_WAID")
PHONE_NUMBER_ID = os.getenv("PHONE_NUMBER_ID")
VERSION = os.getenv("VERSION")

APP_ID = os.getenv("APP_ID")
APP_SECRET = os.getenv("APP_SECRET")

# --------------------------------------------------------------
# Send WhatsApp message
# --------------------------------------------------------------

def get_text_message_input(recipient, text):
    return json.dumps(
        {
            "messaging_product": "whatsapp",
            "recipient_type": "individual",
            "to": recipient,
            "type": "text",
            "text": {"preview_url": False, "body": text},
        }
    )


def send_message(data):
    headers = {
        "Content-type": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}",
    }

    url = f"https://graph.facebook.com/{VERSION}/{PHONE_NUMBER_ID}/messages"

    response = requests.post(url, data=data, headers=headers)
    if response.status_code == 200:
        print("Status:", response.status_code)
        print("Content-type:", response.headers["content-type"])
        print("Body:", response.text)
        return response
    else:
        print(response.status_code)
        print(response.text)
        return response


data = get_text_message_input(
    recipient=RECIPIENT_WAID, text=WA_analytics
)

response = send_message(data)

401
{"error":{"message":"Error validating access token: Session has expired on Monday, 16-Sep-24 00:00:00 PDT. The current time is Monday, 16-Sep-24 06:17:57 PDT.","type":"OAuthException","code":190,"error_subcode":463,"fbtrace_id":"AH9LjflwHLi2aBg3olaaH39"}}
